<A HREF="https://github.com/pday2/teamBlauweKater">tBK GitHub Repository</A><BR>
<A HREF="https://www.americanrhetoric.com/barackobamaspeeches.htm">American Rhetoric Barack Obama Speeches</A>

In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import time # avoid swamping the americanrhetoric website
import os # to check if directory exists and create it if it doesn't
from datetime import datetime # to parse speech date

In [2]:
obamaURL = 'https://www.americanrhetoric.com/barackobamaspeeches.htm'
obamaPage = requests.get(obamaURL, headers={'user-agent': 'Mozilla/5.0'})
soup = BeautifulSoup(obamaPage.content, 'html.parser')

In [57]:
# From main Obama page, get all links to pages with speeches
llist = []

#              url from href                          urls are in <font>                          url cannot be None                    href url must start with speeches/...                    not contain PDFFiles
llist.append([ x.find('a',href=True)['href'] for x in soup.find_all('font', {'face':'Arial'}) if x.find('a',href=True) is not None and x.find('a',href=True)['href'].startswith('speeches/') and 'PDFFiles' not in x.find('a',href=True)['href']])

linklist = llist[0]

<A HREF="https://regexr.com/">RegExr: for testing regex</A><BR>
    There is also <A HREF="https://regex101.com/">Regex101</A>

In [4]:
# Check if Data and associated folders exist for saving speech CSVs to.
# https://djangocentral.com/check-if-a-directory-exists-if-not-create-it/

MYDIRS = ["Data"]

# If folder doesn't exist, then create it.
for MYDIR in MYDIRS:
    CHECK_FOLDER = os.path.isdir(MYDIR)
    if CHECK_FOLDER:
        print(MYDIR, "folder already exists.")
    else:
        os.makedirs(MYDIR)
        print("created folder : ", MYDIR)


Data folder already exists.


<A HREF="https://beautiful-soup-4.readthedocs.io/en/latest/">Beautifulsoup documentation</A><BR>
<A HREF="https://docs.python.org/3/library/re.html">Python regex documentation</A>

In [11]:
# 475 speeches at 3 seconds each is at least 16 minutes 
meta = []
for link in linklist:
    fulllink = 'https://www.americanrhetoric.com/' + link
    try:
        speechPage = requests.get(fulllink, headers={'user-agent': 'Mozilla/5.0'})
    except Exception as e:
        print(e, fulllink)

    pageHTML = BeautifulSoup(speechPage.content, 'html.parser')
    
    #-------------------- maybe remove meta data junk -----------------
    #get meta data - title
    title = re.sub("\r|\n|\t", "", str(pageHTML.title)[7:-8].strip())
    title = title.replace('American Rhetoric: ', '')
    title = title.replace('Barack Obama - ', '')
    print(title)
    
    #get meta data - delivered at
    # This is really ugly
    try:
        startText = '">delivered'
        start = str(pageHTML).find(startText)
        endText = '</font>'
        end = str(pageHTML).find(endText, start)
        delivered = str(pageHTML)[start+len(startText):end].replace('\n', '').replace('\r', '')
    except:
        try:
            startText = '">Delivered'
            start = str(pageHTML).find(startText)
            endText = '</font>'
            end = str(pageHTML).find(endText, start)
            delivered = str(pageHTML)[start+len(startText):end].replace('\n', '').replace('\r', '')
        except:
            print("Rats!")
    #-------------------- maybe remove meta data junk -----------------
    
    #Speech paragraphs are wrapped in <font face="Verdana" size="2">...</font>
    speech = pageHTML.find_all("font", {'face':'Verdana'})

    text = []
    for i in range(len(speech)):
        # Match anything inside an html tag (<>): \<.*?\>
        # could also try [n.get_text(strip=True) for n in speech] which leaves \r and \n
        tmp = re.sub("\r|\n|\t|\<.*?\>", "", str(speech[i]))
        # toss empty paragraphs and the last Book/CDs credit
        if (len(tmp) > 0) and not tmp.startswith('Book/CDs'):
            text.append(tmp)
    
    string = ' '.join(text)
    
    df = pd.DataFrame(text)
    filename = 'Data/' + re.sub("(\w+[\/])", "", link[:-4]) + '.csv'
    textfilename = 'Data/' + re.sub("(\w+[\/])", "", link[:-4]) + '.txt'
    df.to_csv(filename, encoding='utf-8', header=False, index=False)
    # Write a text file
    text_file = open(textfilename, 'w')
    text_file.write(string)
    text_file.close()
    
    temp=[]
    temp.append(title)
    temp.append(delivered[0])
    temp.append(delivered[1:])
    temp.append(filename)
    temp.append(fulllink)
    meta.append(temp)
    time.sleep(3) #be kind!
    


Speech on the Iraq War at Federal Plaza, Chicago
Barack Obama 2004 Democratic National Convention Keynote Address - American Rhetoric
Senate Floor Speech on the Ohio Electoral Vote Counting Process
Knox College Commencement Address
Senate Floor Speech on Rosa Parks (transcript-audio-video)
Senate Floor Speech on the Patriot Act
Senate Floor Speech on the Passing of Coretta Scott King
Senate Floor Speech on Voting Rights Act Renewal
Barack Obama Announces Presidental Exploratory Committee
Announcement of Presidential Candidacy
Selma Voting Rights March Commemoration at Brown Chapel
Senate Floor Speech Supporting Iraq Federalism Amendment
Senate Floor Speech on the War in Iraq After 4 Years
Senate Floor Speech on Immigration
Address at The Woodrow Wilson International Center for Scholars
Speech at the Jefferson-Jackson Dinner
Iowa Caucus Victory Speech
New Hampshire Primary Concession Speech
Address at Ebenezer Baptist Church
Victory Speech in South Carolina
A More Perfect Union (Philade

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.



Senate Floor Speech in Support of Wall Street Bailout Bill
President-Elect Victory Speech
Opening Remarks During First Press Conference as President-Elect
Barack Obama: Taped Remarks to Global Climate Summit
Announces Economic Team
Announces Office of Management and Budget Director and Deputy Director
Announces Office of Management and Budget Director and Deputy Director
Announces Leaders of Economic Recovery Advisory Board
Announces Security Team
Announces Commerce Secretary-Designate
Press Conference with Economic Team (01-05-09)
Speech on the Economy at George Mason University
Announcement of Key Leaders of the National Intelligence Community
Pre-Inauguration Address at the Lincoln Memorial
First Presidential Inaugural Address
Barack Obama: First Presidential Weekly Address (01-24-09)
Al-Arabiya Television Interview
Remarks on Executive Compensation
First Presidential Prime Time Press Conference
Speech at the D.C. Abraham Lincoln Bicentennial Celebration Event (transcript-audio-vid

Address at the Memorial for Joplin Tornado Victims (transcript-audio-video)
On New Defense Personnel (transcript-audio-video)
Address to the Nation on Afghanistan War Troop Reduction (transcript-audio-video)
Address at Carnegie Mellon's National Robotic Center (transcript-audio-video)
Press Conference on Congressional Action to Promote Job Growth (transcript-audio-video)
Speech to the Nation on the Dangers of Default (transcript-audio-video)
Speech on the Debt Compromise (transcript-audio-video)
Speech on the Approved Debt Compromise (transcript-audio-video)
Barack Obama -- Address to the American Legion Conference in Minnesota (transcript-audio-video)
Joint Session of Congress Speech on Jobs (transcript-audio-video)
Rose Garden Speech on Debt Reduction Plan and Economic Growth (transcript-audio-video)
Speech on United Nations Meeting on Libya (transcript-audio-video)
Speech to the UN General Assembly on Palestine Statehood (transcript-audio-video)
Speech at the Martin Luther King Jr. 

CDC Address on Ebola 2014
United Nations General Assembly Address
United Nations Security Concil Remarks 2014
United Nations Meeting on Ebola
Address on Signing Executive Order 13681 at the Consumer Financial Protection Bureau
Ebola Response Update of October 2014
Presser on Republican Midterm Election Victory
Daw Aung San Suu Kyi Joint Press Conference
Young Southeast Asian Leaders Initiative Town Hall
Queensland University Address (text-audio-video)
Barack Obama Immigration Policy Speech to the Nation (text-audio-video)
Barack Obama Remarks on the Ferguson Grand Jury Decision (text-audio-video)
Ebola Vaccine Research NIH Speech
Cuba Policy Changes Address
End of Year Press Conference 2014
America's College Promise Speech at Pellissipi Community College
Federal Trade Commission Address
National Cybersecurity Integration Center Speech
State of the Union Address 2015
Address to the People of India (2015)
National Prayer Breakfast Address 2015
On Force Authorization Request Against ISIL


In [147]:
# Just checking probably delete this once we're happy with the data
text1 = '25 October 2005, Washington, D.C.'
text2 = '4 April 1997'
text1.split(',')[0]
text2.split(',')[1:]
date1 = datetime.strptime(text1.split(',')[0], '%d %B %Y')
print(date1)

2005-10-25 00:00:00


In [148]:
date1.year

2005

In [149]:
date1.month

10

In [150]:
date1.day

25

In [30]:
dfmeta = pd.DataFrame(meta)
dfmeta.to_csv('./Data/genData/metaData.csv', encoding='utf-8', header=False, index=False)

In [17]:
df = pd.DataFrame(linklist)
df.to_csv('./Data/genData/linklist.csv', encoding='utf-8', header=False, index=False)

In [147]:
# I Just went through date_title.csv manually and fixed the speech dates
# American Rhetoric is a mess, the code is a mess and inconsistent
datetitle = pd.read_csv('./Data/genData/date_title.csv')

In [148]:
datetitle=datetitle.dropna() 
datetitle = datetitle.reset_index(drop = True)

In [149]:
datetitle['url'] = ['Data/'+re.sub("(\w+[\/])", "", link[:-4])+'.txt' for link in datetitle['url']]

In [150]:
datetitle['date'] = [ datetime.strptime(day, '%d %b %Y') for day in datetitle['date']]

In [151]:
datetitle.to_csv('./Data/genData/datetitle.csv', index=False)

In [152]:
approval = pd.read_csv('./Data/genData/approvalData.csv', parse_dates=['beg', 'end'])

In [171]:
# Sort for merge_asof
# Sort datetitle by date
datetitle.sort_values(by=['date']);
datetitle.dropna();
# Sort approval by beg
approval.sort_values(by=['beg']);
approval.dropna();

<A HREF="https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf">Pandas cheat sheet</A>

In [181]:
dateTitleAppr = pd.merge_asof(datetitle.sort_values('date'), approval.sort_values('beg'), left_on="date", right_on="beg").dropna()

In [182]:
dateTitleAppr.to_csv('./Data/genData/dateTitleAppr.csv', index=False)

In [9]:
obamaURL = 'https://www.americanrhetoric.com/barackobamaspeeches.htm'
obamaPage = requests.get(obamaURL, headers={'user-agent': 'Mozilla/5.0'})
soup = BeautifulSoup(obamaPage.content, 'html.parser')

### The rest of this is a mess, just trying to get good dates for the AmRhet speeches.  Leaving this here in case it needs to be done again.

In [49]:
# From main Obama page, get all links to pages with speeches
llist = []

#              url from href                          urls are in <font>                          url cannot be None                    href url must start with speeches/...                    not contain PDFFiles
llist.append([ x.find('a',href=True)['href'] for x in soup.find_all('font', {'face':'Arial'}) if x.find('a',href=True) is not None and x.find('a',href=True)['href'].startswith('speeches/') and 'PDFFiles' not in x.find('a',href=True)['href']])

link_list = llist[0]

In [43]:
# From main Obama page, get dates
#              url from href                          urls are in <font>                          url cannot be None                    href url must start with speeches/...                    not contain PDFFiles
candidates=[ x.text for x in soup.find_all('font', {'color':'#000048'}) if x is not None ]
date_list = [text for text in candidates if text[7:11].isdigit()]

In [56]:
candidates2=[ x.find('a',href=True)['href'] for x in soup.find_all('font', {'color':'#000048'}) if x.find('a',href=True)['href'] is not None ]
linky_list = [text for text in candidates2 if not text[7:11].isdigit()]

TypeError: 'NoneType' object is not subscriptable

In [58]:
tr=[x for x in soup.find_all('tr')]

In [62]:
td_in_tr=[x.find_all('td') for x in tr]

In [111]:
date_link = [x[0:2] for x in td_in_tr]

In [112]:
date_link=date_link[34:516]

In [141]:
[' '.join(x) for x in date_link]


TypeError: sequence item 0: expected str instance, Tag found

In [144]:
f = open("amRhetHTML_dl.txt")
text = f.read()
f.close()
print(text)

    <tr>
      <td style="padding-left: 11px" width="148" height="37" bgcolor="#E2EBFE" align="left">
      <b><font size="4" face="Tahoma">02 Oct 2002</font></b></td>
      <td style="padding-left: 5px" width="473" height="37" bgcolor="#FFFFFF" align="left">
      <font face="Arial">
		<a style="text-decoration: none" href="https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamairaqwarspeechfederalplaza.htm">
		<font size="4" color="#000048">Federal Plaza Address Opposing the War in Iraq</font></a></font></td>
      <td width="83" height="37" bgcolor="#800000" align="center">
		<font face="Arial">
<a href="https://www.americanrhetoric.com/mp3clips/barackobama/barackobamafederalplaza.mp3">
<font color="#FFFFFF"><u>mp3</u></font></a></font></td>
      <td width="78" height="37" align="center">
      	&nbsp;</td>
    </tr>
    <tr>
      <td style="padding-left: 11px" width="148" height="37" bgcolor="#E2EBFE" align="left">
      <b><font size="4" face="Tahoma">27 Jul 2004<

In [156]:
results = []
datestart = '<font size="4" face="Tahoma"'
dateend = '</font>'
linkstart = '''<a style="text-decoration: none" href="'''
linkend = '.htm">'
textstart = '<font size="4" color="#000048">'
textend = '</font>'
#string.find(value, start, end) 
start = 0
for i in range(1500):
    try:
        dstart=text.find(datestart, start)
        dend=text.find(dateend, dstart+len(datestart))
        date=text[dstart+len(datestart)+1:dend]
    except:
        date = 'xxx'
    try:
        lstart=text.find(linkstart, dend+len(dateend))
        lend=text.find(linkend, lstart+len(linkstart))
        link=text[lstart+len(linkstart):lend]
    except:
        link = 'yyy'
    try:
        tstart=text.find(textstart, lend+len(linkend))
        tend=text.find(textend, tstart+len(textstart))
        ltext=text[tstart+len(textstart):tend]
    except:
        ltext = 'zzz'
    line = [date, link, ltext]
    results.append(line)
    start = tend+10

In [157]:
results

[['02 Oct 2002',
  'https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamairaqwarspeechfederalplaza',
  'Federal Plaza Address Opposing the War in Iraq'],
 ['27 Jul 2004',
  'https://www.americanrhetoric.co./Data/speeches/convention2004/barackobama2004dnc',
  'Democratic National Convention Keynote Speech'],
 ['06 Jan 2005',
  'https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamasenatespeechonohioelectoralvotes',
  'Senate Speech on Ohio Electoral Vote'],
 ['color="#000048">04 Jun 2005',
  'https://www.americanrhetoric.co./Data/speeches/barackobamaknoxcollege',
  'Senate Speech Honoring the Life of Rosa Parks'],
 ['color="#000048">15 Dec 2005',
  'https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamasenatefloorspeechpatriotact',
  'Senate Speech on the PATRIOT Act'],
 ['color="#000048">31 Jan 2006',
  'https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamasenatespeechoncorettascottkingpassing',
  'Senate Speech Honoring

In [163]:
results2 = [[result[0].replace('color="#000048">', ''), result[1], result[2]] for result in results]

In [164]:
results2

[['02 Oct 2002',
  'https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamairaqwarspeechfederalplaza',
  'Federal Plaza Address Opposing the War in Iraq'],
 ['27 Jul 2004',
  'https://www.americanrhetoric.co./Data/speeches/convention2004/barackobama2004dnc',
  'Democratic National Convention Keynote Speech'],
 ['06 Jan 2005',
  'https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamasenatespeechonohioelectoralvotes',
  'Senate Speech on Ohio Electoral Vote'],
 ['04 Jun 2005',
  'https://www.americanrhetoric.co./Data/speeches/barackobamaknoxcollege',
  'Senate Speech Honoring the Life of Rosa Parks'],
 ['15 Dec 2005',
  'https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamasenatefloorspeechpatriotact',
  'Senate Speech on the PATRIOT Act'],
 ['31 Jan 2006',
  'https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamasenatespeechoncorettascottkingpassing',
  'Senate Speech Honoring the Life of \n\t\tCoretta Scott King'],
 ['20 J

In [165]:
results3 = [[result[0], result[1], result[2].replace('\n', '')] for result in results2]
results4 = [[result[0], result[1], result[2].replace('\t', '')] for result in results3]

In [166]:
results4

[['02 Oct 2002',
  'https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamairaqwarspeechfederalplaza',
  'Federal Plaza Address Opposing the War in Iraq'],
 ['27 Jul 2004',
  'https://www.americanrhetoric.co./Data/speeches/convention2004/barackobama2004dnc',
  'Democratic National Convention Keynote Speech'],
 ['06 Jan 2005',
  'https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamasenatespeechonohioelectoralvotes',
  'Senate Speech on Ohio Electoral Vote'],
 ['04 Jun 2005',
  'https://www.americanrhetoric.co./Data/speeches/barackobamaknoxcollege',
  'Senate Speech Honoring the Life of Rosa Parks'],
 ['15 Dec 2005',
  'https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamasenatefloorspeechpatriotact',
  'Senate Speech on the PATRIOT Act'],
 ['31 Jan 2006',
  'https://www.americanrhetoric.co./Data/speeches/barackobama/barackobamasenatespeechoncorettascottkingpassing',
  'Senate Speech Honoring the Life of Coretta Scott King'],
 ['20 Jul 200

In [169]:
results5 = [[result[0], result[1].replace('https://www.americanrhetoric.co./Data/speeches/barackobama/', ''), result[2]] for result in results4]

In [171]:
results6 = [[result[0], result[1].replace('https://www.americanrhetoric.co./Data/speeches/convention2004/', ''), result[2]] for result in results5]

In [172]:
results7 = [[result[0], result[1].replace('https://www.americanrhetoric.co./Data/speeches/barackobama/', ''), result[2]] for result in results6]

In [174]:
results8 = [[result[0], result[1].replace('https://www.americanrhetoric.co./Data/speeches/', ''), result[2]] for result in results7]

In [179]:
results9 = results8 = [[result[0], result[1]+'.txt', result[2]] for result in results8]

In [180]:
datedata=pd.DataFrame(results9)

In [181]:
datedata.to_csv("AmRhet_date_data.csv", index=False, header=False)